In [64]:
import math
import networkx as nx
import igraph as ig
import leidenalg as la
from copy import deepcopy
from pelote import read_graphology_json, triangular_strength, filter_edges
from pelote.graph import union_of_maximum_spanning_trees
from ipysigma import Sigma, SigmaGrid
from fog.metrics import sparse_cosine_similarity
from fog.metrics.utils import intersection_size
from collections import defaultdict, Counter

Sigma.set_defaults(max_categorical_colors=25)

# Self similarity using communities

## Leiden helpers

In [105]:
def leiden_modularity(g: nx.Graph, weighted: bool = False):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [float(w) for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, la.ModularityVertexPartition, weights=weights)
    return {n: m for n, m in zip(g, partition._membership)}

In [106]:
def leiden_cpm(g: nx.Graph, weighted: bool = False, resolution: float = 0.05):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [float(w) for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, la.CPMVertexPartition, weights=weights, resolution_parameter=resolution)
    return {n: m for n, m in zip(g, partition._membership)}

In [138]:
def leiden_abstract(g: nx.Graph, method, weighted: bool = False):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [float(w) for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, method, weights=weights)
    return {n: m for n, m in zip(g, partition._membership)}

In [139]:
def leiden_abstract_resolution(g: nx.Graph, method, weighted: bool = False, resolution: float = 0.05):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [float(w) for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, method, weights=weights, resolution_parameter=resolution)
    return {n: m for n, m in zip(g, partition._membership)}

## Les misérables

In [46]:
# Here we project by considering each node as a weighted vector of neighboring communities
def selfcomsim(g, direction: str = 'all', keep_own_community: bool = True) -> nx.Graph:
    partition = leiden_modularity(g)
    projection = nx.Graph()

    neighbor_fn = g.neighbors

    if direction == 'out':
        neighbor_fn = g.successors
    elif direction == 'in':
        neighbor_fn = g.predecessors

    vectors = {}

    for node, attr in g.nodes.data():
        projection.add_node(node, **attr)
        vector = Counter()
        own_community = partition[node]

        # NOTE: should we drop own community?
        for neighbor in neighbor_fn(node):
            community = partition[neighbor]

            if not keep_own_community and community == own_community:
                continue    
            
            vector[community] += 1

        vectors[node] = vector

    nodes = list(g)

    for i in range(len(nodes)):
        n1 = nodes[i]
        v1 = vectors[n1]
        
        for j in range(i + 1, len(nodes)):
            n2 = nodes[j]  
            v2 = vectors[n2]

            w = sparse_cosine_similarity(v1, v2)

            if w > 0:
                projection.add_edge(n1, n2, weight=w)

    return projection

In [5]:
miserables = nx.les_miserables_graph()

In [14]:
Sigma(miserables, node_color=leiden_modularity(miserables), node_size=miserables.degree)

Sigma(nx.Graph with 77 nodes and 254 edges)

In [50]:
miserables_projection = selfcomsim(miserables, keep_own_community=True)

In [48]:
Sigma(miserables_projection, node_size=miserables.degree, node_color=leiden_modularity(miserables_projection))

Sigma(nx.Graph with 77 nodes and 425 edges)

In [49]:
SigmaGrid(miserables, node_size=miserables.degree, views=[
    {"name": "Normal", "node_color": leiden_modularity(miserables)},
    {"name": "Projection", "node_color": leiden_modularity(miserables_projection)}
])

## EuroSiS

In [24]:
eurosis = read_graphology_json('./eurosis.json')

In [53]:
eurosis_projection = selfcomsim(eurosis, keep_own_community=True)

In [54]:
SigmaGrid(eurosis, node_size=eurosis.degree, views=[
    {"name": "Normal", "node_color": leiden_modularity(eurosis)},
    {"name": "Projection", "node_color": leiden_modularity(eurosis_projection)}
])

## Polarisation

In [55]:
corpus = read_graphology_json('./NETWORK_CorpusMedia_DEFACTO_medialab_SciencesPo_V1.json')

for node, attr in corpus.nodes.data():
    del attr['x']
    del attr['y']
    del attr['color']

In [126]:
corpus_projection = selfcomsim(corpus, direction='out', keep_own_community=False)

In [127]:
SigmaGrid(corpus, node_size=corpus.degree, views=[
    {"name": "Normal", "node_color": leiden_modularity(corpus)},
    {"name": "Projection", "node_color": leiden_modularity(corpus_projection)}
])

In [143]:
SigmaGrid(
    corpus,
    node_size=corpus.degree,
    views=[
        # {"name": "Modularity", "node_color": leiden_abstract(corpus, la.ModularityVertexPartition)},
        {"name": "CPM", "node_color": leiden_abstract_resolution(corpus, la.CPMVertexPartition, resolution=0.5)},
        {"name": "Surprise", "node_color": leiden_abstract(corpus, la.SurpriseVertexPartition)},
        {"name": "Significance", "node_color": leiden_abstract(corpus, la.SignificanceVertexPartition)},
        {"name": "RBConfig", "node_color": leiden_abstract(corpus, la.RBConfigurationVertexPartition)},
        {"name": "RBER", "node_color": leiden_abstract(corpus, la.RBERVertexPartition)}
    ]
)

## Self-similarity projection of induced community graph

In [107]:
# NOTE: here we project the weighted community graph instead
def comselfsim(g, direction: str = 'all', resolution: float = 0.05):
    partition = leiden_cpm(g, resolution=resolution)
    projection = nx.Graph()

    neighbor_fn = g.neighbors

    if direction == 'out':
        neighbor_fn = g.successors
    elif direction == 'in':
        neighbor_fn = g.predecessors

    community_vectors = defaultdict(Counter)

    for node in g:
        community = partition[node]

        for neighbor in neighbor_fn(node):
            neighbor_community = partition[neighbor]

            community_vectors[community][neighbor_community] += 1

    communities = list(community_vectors)

    for i in range(len(communities)):        
        c1 = communities[i]
        v1 = community_vectors[c1]
        for j in range(i + 1, len(communities)):
            c2 = communities[j]
            v2 = community_vectors[c2]
            w = sparse_cosine_similarity(v1, v2)

            if w > 0:
                projection.add_edge(c1, c2, weight=w)

    projection_partition = leiden_modularity(projection, weighted=True)

    return {node: projection_partition.get(partition[node], None) for node in g}

In [109]:
def test(g, direction: str = 'all', resolution: float = 0.05):
    return Sigma(g, node_color=comselfsim(g, direction=direction, resolution=resolution), node_size=g.degree)

In [124]:
test(corpus, resolution=50, direction='out')

Sigma(nx.DiGraph with 732 nodes and 27,556 edges)